In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
#定义超参数
input_size = 28  #图像的总尺寸28*28
num_classes = 10  #标签的种类数
num_epochs = 3   #训练的总循环周期
batch_size = 64  #一个批次的大小，64张照片
 
#训练集
train_dataset = datasets.MNIST(root='./data',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
 
#测试集
test_dataset = datasets.MNIST(root='./data',
                              train=False,
                              transform=transforms.ToTensor(),)
 
#构建batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                       batch_size=batch_size,
                                       shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(       #输入大小（1，28，28）
            nn.Conv2d(
                in_channels=1,           #灰度图
                out_channels=16,         #要得到多少个特征图
                kernel_size=5,           #卷积核大小
                stride=1,                #步长
                padding=2,               #如果希望卷积后大小和原来一样，需要设置padding=（kernel_size-1）/2 if stride=1
            ),                           #输出的特征图为（16，28，28）
            nn.ReLU(),                   #relu层
            nn.MaxPool2d(kernel_size=2),#进行池化操作（2*2区域），输出结果为：（16，14，14）
        )
        self.conv2 = nn.Sequential(      #下一个套餐的输入 （16，14，14）
        nn.Conv2d(16, 32, 5, 1, 2),       #输出（32，14，14）
        nn.ReLU(),                       #relu层
        nn.MaxPool2d(2),                 #输出（32，7，7）
        )
        self.out = nn.Linear(32 * 7 * 7, 10)  #全连接层得到的结果
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)         #faltten操作，结果为：(batch_size, 32*7*7)
        output = self.out(x)
        return output

In [4]:
def accuracy(predictions,lables):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(lables.data.view_as(pred)).sum()
    return rights, len(lables)

In [5]:
#实例化
net = CNN()
#损失函数
criterion = nn.CrossEntropyLoss()
#优化器
optimizer = optim.Adam(net.parameters(), lr=0.001)             #定义优化器，普通的随机梯度下降算法
 
#开始训练循环
for epoch in range(num_epochs):
    #当前epoch的结果保存下来
    train_rights = []
    
    for batch_idx, (data, target) in enumerate(train_loader):   #针对容器中的每一个批进行循环  
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)
        
        if batch_idx % 100 == 0:
            
            net.eval()
            val_rights = []
            
            for (data, target) in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
                
        #准确率计算
        train_r = (sum([tup[0] for tup in train_rights]),sum([tup[1] for tup in train_rights]))
        val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
        
        print('当前epoch：{} [{}/{} ({:.0f}%)]\t损失: {:.6f}\t训练集准确率: {:.2f}%\t测试集正确率: {:.2f}'.format(
             epoch, batch_idx * batch_size, len(train_loader.dataset),
             100. * batch_idx / len(train_loader),
             loss.data,
             100. * train_r[0].numpy() / train_r[1],
             100. * val_r[0].numpy() / val_r[1]))

当前epoch：0 [0/60000 (0%)]	损失: 2.297984	训练集准确率: 7.81%	测试集正确率: 12.40
当前epoch：0 [64/60000 (0%)]	损失: 2.297592	训练集准确率: 10.16%	测试集正确率: 12.40
当前epoch：0 [128/60000 (0%)]	损失: 2.300313	训练集准确率: 9.90%	测试集正确率: 12.40
当前epoch：0 [192/60000 (0%)]	损失: 2.233591	训练集准确率: 10.55%	测试集正确率: 12.40
当前epoch：0 [256/60000 (0%)]	损失: 2.254881	训练集准确率: 10.62%	测试集正确率: 12.40
当前epoch：0 [320/60000 (1%)]	损失: 2.219146	训练集准确率: 11.98%	测试集正确率: 12.40
当前epoch：0 [384/60000 (1%)]	损失: 2.212116	训练集准确率: 13.84%	测试集正确率: 12.40
当前epoch：0 [448/60000 (1%)]	损失: 2.255970	训练集准确率: 14.84%	测试集正确率: 12.40
当前epoch：0 [512/60000 (1%)]	损失: 2.152119	训练集准确率: 16.49%	测试集正确率: 12.40
当前epoch：0 [576/60000 (1%)]	损失: 2.149921	训练集准确率: 17.03%	测试集正确率: 12.40
当前epoch：0 [640/60000 (1%)]	损失: 2.093802	训练集准确率: 18.75%	测试集正确率: 12.40
当前epoch：0 [704/60000 (1%)]	损失: 2.089460	训练集准确率: 20.44%	测试集正确率: 12.40
当前epoch：0 [768/60000 (1%)]	损失: 2.029830	训练集准确率: 22.36%	测试集正确率: 12.40
当前epoch：0 [832/60000 (1%)]	损失: 2.091686	训练集准确率: 23.33%	测试集正确率: 12.40
当前epoch：0 [896/60000 (1%)]	损失: 1.966504

当前epoch：0 [8000/60000 (13%)]	损失: 0.277414	训练集准确率: 78.25%	测试集正确率: 91.44
当前epoch：0 [8064/60000 (13%)]	损失: 0.321776	训练集准确率: 78.33%	测试集正确率: 91.44
当前epoch：0 [8128/60000 (14%)]	损失: 0.075993	训练集准确率: 78.49%	测试集正确率: 91.44
当前epoch：0 [8192/60000 (14%)]	损失: 0.235625	训练集准确率: 78.62%	测试集正确率: 91.44
当前epoch：0 [8256/60000 (14%)]	损失: 0.349687	训练集准确率: 78.71%	测试集正确率: 91.44
当前epoch：0 [8320/60000 (14%)]	损失: 0.151843	训练集准确率: 78.84%	测试集正确率: 91.44
当前epoch：0 [8384/60000 (14%)]	损失: 0.141523	训练集准确率: 78.98%	测试集正确率: 91.44
当前epoch：0 [8448/60000 (14%)]	损失: 0.300653	训练集准确率: 79.05%	测试集正确率: 91.44
当前epoch：0 [8512/60000 (14%)]	损失: 0.126692	训练集准确率: 79.19%	测试集正确率: 91.44
当前epoch：0 [8576/60000 (14%)]	损失: 0.281096	训练集准确率: 79.28%	测试集正确率: 91.44
当前epoch：0 [8640/60000 (14%)]	损失: 0.174002	训练集准确率: 79.39%	测试集正确率: 91.44
当前epoch：0 [8704/60000 (14%)]	损失: 0.226750	训练集准确率: 79.48%	测试集正确率: 91.44
当前epoch：0 [8768/60000 (15%)]	损失: 0.129490	训练集准确率: 79.60%	测试集正确率: 91.44
当前epoch：0 [8832/60000 (15%)]	损失: 0.107762	训练集准确率: 79.71%	测试集正确率: 91.44
当前epoc

当前epoch：0 [15680/60000 (26%)]	损失: 0.061303	训练集准确率: 86.13%	测试集正确率: 95.46
当前epoch：0 [15744/60000 (26%)]	损失: 0.101580	训练集准确率: 86.18%	测试集正确率: 95.46
当前epoch：0 [15808/60000 (26%)]	损失: 0.170660	训练集准确率: 86.21%	测试集正确率: 95.46
当前epoch：0 [15872/60000 (26%)]	损失: 0.069582	训练集准确率: 86.26%	测试集正确率: 95.46
当前epoch：0 [15936/60000 (27%)]	损失: 0.156602	训练集准确率: 86.30%	测试集正确率: 95.46
当前epoch：0 [16000/60000 (27%)]	损失: 0.189811	训练集准确率: 86.34%	测试集正确率: 95.46
当前epoch：0 [16064/60000 (27%)]	损失: 0.059866	训练集准确率: 86.39%	测试集正确率: 95.46
当前epoch：0 [16128/60000 (27%)]	损失: 0.199856	训练集准确率: 86.41%	测试集正确率: 95.46
当前epoch：0 [16192/60000 (27%)]	损失: 0.188496	训练集准确率: 86.44%	测试集正确率: 95.46
当前epoch：0 [16256/60000 (27%)]	损失: 0.046954	训练集准确率: 86.48%	测试集正确率: 95.46
当前epoch：0 [16320/60000 (27%)]	损失: 0.268444	训练集准确率: 86.51%	测试集正确率: 95.46
当前epoch：0 [16384/60000 (27%)]	损失: 0.105990	训练集准确率: 86.55%	测试集正确率: 95.46
当前epoch：0 [16448/60000 (27%)]	损失: 0.141025	训练集准确率: 86.57%	测试集正确率: 95.46
当前epoch：0 [16512/60000 (28%)]	损失: 0.262509	训练集准确率: 86.58%	测试集正确率

当前epoch：0 [23040/60000 (38%)]	损失: 0.083213	训练集准确率: 89.23%	测试集正确率: 96.34
当前epoch：0 [23104/60000 (38%)]	损失: 0.096777	训练集准确率: 89.25%	测试集正确率: 96.34
当前epoch：0 [23168/60000 (39%)]	损失: 0.079362	训练集准确率: 89.27%	测试集正确率: 96.34
当前epoch：0 [23232/60000 (39%)]	损失: 0.073509	训练集准确率: 89.29%	测试集正确率: 96.34
当前epoch：0 [23296/60000 (39%)]	损失: 0.110571	训练集准确率: 89.31%	测试集正确率: 96.34
当前epoch：0 [23360/60000 (39%)]	损失: 0.065500	训练集准确率: 89.34%	测试集正确率: 96.34
当前epoch：0 [23424/60000 (39%)]	损失: 0.122363	训练集准确率: 89.35%	测试集正确率: 96.34
当前epoch：0 [23488/60000 (39%)]	损失: 0.079725	训练集准确率: 89.38%	测试集正确率: 96.34
当前epoch：0 [23552/60000 (39%)]	损失: 0.073343	训练集准确率: 89.41%	测试集正确率: 96.34
当前epoch：0 [23616/60000 (39%)]	损失: 0.031993	训练集准确率: 89.43%	测试集正确率: 96.34
当前epoch：0 [23680/60000 (39%)]	损失: 0.118198	训练集准确率: 89.45%	测试集正确率: 96.34
当前epoch：0 [23744/60000 (40%)]	损失: 0.080089	训练集准确率: 89.48%	测试集正确率: 96.34
当前epoch：0 [23808/60000 (40%)]	损失: 0.062295	训练集准确率: 89.50%	测试集正确率: 96.34
当前epoch：0 [23872/60000 (40%)]	损失: 0.048547	训练集准确率: 89.53%	测试集正确率

当前epoch：0 [30848/60000 (51%)]	损失: 0.086142	训练集准确率: 91.07%	测试集正确率: 96.33
当前epoch：0 [30912/60000 (51%)]	损失: 0.039096	训练集准确率: 91.09%	测试集正确率: 96.33
当前epoch：0 [30976/60000 (52%)]	损失: 0.142743	训练集准确率: 91.10%	测试集正确率: 96.33
当前epoch：0 [31040/60000 (52%)]	损失: 0.093046	训练集准确率: 91.11%	测试集正确率: 96.33
当前epoch：0 [31104/60000 (52%)]	损失: 0.168196	训练集准确率: 91.12%	测试集正确率: 96.33
当前epoch：0 [31168/60000 (52%)]	损失: 0.053855	训练集准确率: 91.14%	测试集正确率: 96.33
当前epoch：0 [31232/60000 (52%)]	损失: 0.047524	训练集准确率: 91.16%	测试集正确率: 96.33
当前epoch：0 [31296/60000 (52%)]	损失: 0.031202	训练集准确率: 91.17%	测试集正确率: 96.33
当前epoch：0 [31360/60000 (52%)]	损失: 0.060993	训练集准确率: 91.19%	测试集正确率: 96.33
当前epoch：0 [31424/60000 (52%)]	损失: 0.037130	训练集准确率: 91.20%	测试集正确率: 96.33
当前epoch：0 [31488/60000 (52%)]	损失: 0.063187	训练集准确率: 91.21%	测试集正确率: 96.33
当前epoch：0 [31552/60000 (53%)]	损失: 0.153503	训练集准确率: 91.22%	测试集正确率: 96.33
当前epoch：0 [31616/60000 (53%)]	损失: 0.141471	训练集准确率: 91.23%	测试集正确率: 96.33
当前epoch：0 [31680/60000 (53%)]	损失: 0.046379	训练集准确率: 91.24%	测试集正确率

...

当前epoch：2 [24192/60000 (40%)]	损失: 0.030100	训练集准确率: 98.84%	测试集正确率: 98.73
当前epoch：2 [24256/60000 (40%)]	损失: 0.067026	训练集准确率: 98.84%	测试集正确率: 98.73
当前epoch：2 [24320/60000 (41%)]	损失: 0.038090	训练集准确率: 98.84%	测试集正确率: 98.73
当前epoch：2 [24384/60000 (41%)]	损失: 0.109536	训练集准确率: 98.83%	测试集正确率: 98.73
当前epoch：2 [24448/60000 (41%)]	损失: 0.006021	训练集准确率: 98.84%	测试集正确率: 98.73
当前epoch：2 [24512/60000 (41%)]	损失: 0.006145	训练集准确率: 98.84%	测试集正确率: 98.73
当前epoch：2 [24576/60000 (41%)]	损失: 0.041019	训练集准确率: 98.84%	测试集正确率: 98.73
当前epoch：2 [24640/60000 (41%)]	损失: 0.008811	训练集准确率: 98.84%	测试集正确率: 98.73
当前epoch：2 [24704/60000 (41%)]	损失: 0.042362	训练集准确率: 98.84%	测试集正确率: 98.73
当前epoch：2 [24768/60000 (41%)]	损失: 0.073281	训练集准确率: 98.84%	测试集正确率: 98.73
当前epoch：2 [24832/60000 (41%)]	损失: 0.059359	训练集准确率: 98.84%	测试集正确率: 98.73
当前epoch：2 [24896/60000 (41%)]	损失: 0.026120	训练集准确率: 98.83%	测试集正确率: 98.73
当前epoch：2 [24960/60000 (42%)]	损失: 0.051423	训练集准确率: 98.83%	测试集正确率: 98.73
当前epoch：2 [25024/60000 (42%)]	损失: 0.012477	训练集准确率: 98.83%	测试集正确率

当前epoch：2 [31872/60000 (53%)]	损失: 0.051580	训练集准确率: 98.83%	测试集正确率: 98.76
当前epoch：2 [31936/60000 (53%)]	损失: 0.080504	训练集准确率: 98.83%	测试集正确率: 98.76
当前epoch：2 [32000/60000 (53%)]	损失: 0.028713	训练集准确率: 98.82%	测试集正确率: 98.90
当前epoch：2 [32064/60000 (53%)]	损失: 0.009838	训练集准确率: 98.83%	测试集正确率: 98.90
当前epoch：2 [32128/60000 (54%)]	损失: 0.005283	训练集准确率: 98.83%	测试集正确率: 98.90
当前epoch：2 [32192/60000 (54%)]	损失: 0.038598	训练集准确率: 98.83%	测试集正确率: 98.90
当前epoch：2 [32256/60000 (54%)]	损失: 0.102613	训练集准确率: 98.83%	测试集正确率: 98.90
当前epoch：2 [32320/60000 (54%)]	损失: 0.021826	训练集准确率: 98.83%	测试集正确率: 98.90
当前epoch：2 [32384/60000 (54%)]	损失: 0.042012	训练集准确率: 98.83%	测试集正确率: 98.90
当前epoch：2 [32448/60000 (54%)]	损失: 0.054495	训练集准确率: 98.83%	测试集正确率: 98.90
当前epoch：2 [32512/60000 (54%)]	损失: 0.124735	训练集准确率: 98.82%	测试集正确率: 98.90
当前epoch：2 [32576/60000 (54%)]	损失: 0.025368	训练集准确率: 98.82%	测试集正确率: 98.90
当前epoch：2 [32640/60000 (54%)]	损失: 0.072121	训练集准确率: 98.82%	测试集正确率: 98.90
当前epoch：2 [32704/60000 (54%)]	损失: 0.024805	训练集准确率: 98.82%	测试集正确率

当前epoch：2 [39552/60000 (66%)]	损失: 0.003854	训练集准确率: 98.84%	测试集正确率: 98.95
当前epoch：2 [39616/60000 (66%)]	损失: 0.027852	训练集准确率: 98.84%	测试集正确率: 98.95
当前epoch：2 [39680/60000 (66%)]	损失: 0.050886	训练集准确率: 98.84%	测试集正确率: 98.95
当前epoch：2 [39744/60000 (66%)]	损失: 0.003312	训练集准确率: 98.84%	测试集正确率: 98.95
当前epoch：2 [39808/60000 (66%)]	损失: 0.002761	训练集准确率: 98.84%	测试集正确率: 98.95
当前epoch：2 [39872/60000 (66%)]	损失: 0.018956	训练集准确率: 98.84%	测试集正确率: 98.95
当前epoch：2 [39936/60000 (67%)]	损失: 0.032020	训练集准确率: 98.84%	测试集正确率: 98.95
当前epoch：2 [40000/60000 (67%)]	损失: 0.070036	训练集准确率: 98.84%	测试集正确率: 98.95
当前epoch：2 [40064/60000 (67%)]	损失: 0.015592	训练集准确率: 98.84%	测试集正确率: 98.95
当前epoch：2 [40128/60000 (67%)]	损失: 0.012696	训练集准确率: 98.84%	测试集正确率: 98.95
当前epoch：2 [40192/60000 (67%)]	损失: 0.002866	训练集准确率: 98.84%	测试集正确率: 98.95
当前epoch：2 [40256/60000 (67%)]	损失: 0.011785	训练集准确率: 98.85%	测试集正确率: 98.95
当前epoch：2 [40320/60000 (67%)]	损失: 0.004232	训练集准确率: 98.85%	测试集正确率: 98.95
当前epoch：2 [40384/60000 (67%)]	损失: 0.009819	训练集准确率: 98.85%	测试集正确率

当前epoch：2 [47104/60000 (78%)]	损失: 0.090308	训练集准确率: 98.81%	测试集正确率: 98.84
当前epoch：2 [47168/60000 (79%)]	损失: 0.038831	训练集准确率: 98.81%	测试集正确率: 98.84
当前epoch：2 [47232/60000 (79%)]	损失: 0.029768	训练集准确率: 98.81%	测试集正确率: 98.84
当前epoch：2 [47296/60000 (79%)]	损失: 0.015955	训练集准确率: 98.81%	测试集正确率: 98.84
当前epoch：2 [47360/60000 (79%)]	损失: 0.007199	训练集准确率: 98.81%	测试集正确率: 98.84
当前epoch：2 [47424/60000 (79%)]	损失: 0.018286	训练集准确率: 98.82%	测试集正确率: 98.84
当前epoch：2 [47488/60000 (79%)]	损失: 0.008984	训练集准确率: 98.82%	测试集正确率: 98.84
当前epoch：2 [47552/60000 (79%)]	损失: 0.020831	训练集准确率: 98.82%	测试集正确率: 98.84
当前epoch：2 [47616/60000 (79%)]	损失: 0.038691	训练集准确率: 98.82%	测试集正确率: 98.84
当前epoch：2 [47680/60000 (79%)]	损失: 0.011274	训练集准确率: 98.82%	测试集正确率: 98.84
当前epoch：2 [47744/60000 (80%)]	损失: 0.005130	训练集准确率: 98.82%	测试集正确率: 98.84
当前epoch：2 [47808/60000 (80%)]	损失: 0.035139	训练集准确率: 98.82%	测试集正确率: 98.84
当前epoch：2 [47872/60000 (80%)]	损失: 0.017562	训练集准确率: 98.82%	测试集正确率: 98.84
当前epoch：2 [47936/60000 (80%)]	损失: 0.017455	训练集准确率: 98.82%	测试集正确率

当前epoch：2 [54400/60000 (91%)]	损失: 0.010065	训练集准确率: 98.82%	测试集正确率: 99.02
当前epoch：2 [54464/60000 (91%)]	损失: 0.124508	训练集准确率: 98.82%	测试集正确率: 99.02
当前epoch：2 [54528/60000 (91%)]	损失: 0.003424	训练集准确率: 98.82%	测试集正确率: 99.02
当前epoch：2 [54592/60000 (91%)]	损失: 0.063405	训练集准确率: 98.81%	测试集正确率: 99.02
当前epoch：2 [54656/60000 (91%)]	损失: 0.087170	训练集准确率: 98.81%	测试集正确率: 99.02
当前epoch：2 [54720/60000 (91%)]	损失: 0.087518	训练集准确率: 98.81%	测试集正确率: 99.02
当前epoch：2 [54784/60000 (91%)]	损失: 0.010848	训练集准确率: 98.81%	测试集正确率: 99.02
当前epoch：2 [54848/60000 (91%)]	损失: 0.008481	训练集准确率: 98.81%	测试集正确率: 99.02
当前epoch：2 [54912/60000 (91%)]	损失: 0.003310	训练集准确率: 98.82%	测试集正确率: 99.02
当前epoch：2 [54976/60000 (92%)]	损失: 0.017003	训练集准确率: 98.82%	测试集正确率: 99.02
当前epoch：2 [55040/60000 (92%)]	损失: 0.008015	训练集准确率: 98.82%	测试集正确率: 99.02
当前epoch：2 [55104/60000 (92%)]	损失: 0.008200	训练集准确率: 98.82%	测试集正确率: 99.02
当前epoch：2 [55168/60000 (92%)]	损失: 0.002322	训练集准确率: 98.82%	测试集正确率: 99.02
当前epoch：2 [55232/60000 (92%)]	损失: 0.013155	训练集准确率: 98.82%	测试集正确率